02で前処理をしたデータの読み込みと03で学習したモデルを使って推論を行うためのnotebookです。  

## 必要なライブラリのimport

In [43]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import pickle

In [67]:
# 予測したいデータの読み込み
#test = pd.read_csv('../data/processed/processed20240614_test.csv')
test = pd.read_csv('../data/processed/processed20240621_test.csv')
test_flpr = pd.read_csv('../data/processed/test_with_outlier_flag_probability.csv')

del test['first_active_month']

test = pd.concat([test,test_flpr[['first_active_month','outlier_flag', 'outlier_probability']]], axis = 1)

In [45]:
test.head()

,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,hist_purchase_date_y,hist_category_1_sum,...,category_1_installments_max,category_1_installments_std,rate_new/hist_purchase_date_max,rate_new/hist_month_diff_mean,rate_new/hist_purchase_amount_sum,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean,first_active_month,outlier_flag,outlier_probability
0,C_ID_0ab67a22ab,3,3,1,306.0,24.0,1.513186e+09,-0.476731,1.503441e+09,13.0,...,1.666667,NaN,1.004401,1.000000,0.140001,-6.388255,1.120009,0.501563,0,2.074851e-03
1,C_ID_130fd0cbdd,2,3,0,396.0,1.0,1.494079e+09,-0.731881,1.494079e+09,1.0,...,1.285714,0.202031,1.020192,-inf,8.121995,0.691844,0.902444,0.194557,0,9.908335e-06
2,C_ID_b709037bc5,5,1,1,184.0,4.0,1.517593e+09,0.214624,1.515681e+09,0.0,...,10.000000,6.363961,1.002210,0.500000,0.033132,0.358015,0.066264,0.917728,0,1.796332e-06
3,C_ID_d27d835a9f,2,1,0,62.0,NaN,NaN,NaN,NaN,NaN,...,12.000000,7.149635,NaN,NaN,NaN,NaN,NaN,1.333892,0,3.490204e-11
4,C_ID_2b5e3df5c2,5,1,1,793.0,23.0,1.514992e+09,-0.596643,1.491330e+09,0.0,...,0.833333,NaN,1.005639,3.833333,-0.775375,39.511246,-2.972271,-1.159684,0,3.157762e-16


In [68]:
#モデルによるq1q2q3
del test['outlier_probability']


In [69]:
test1 = test.query('outlier_flag == 1')
test = test.query('outlier_flag == 0')

In [47]:
test.head()

,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,hist_purchase_date_y,hist_category_1_sum,...,category_1_installments_min,category_1_installments_max,category_1_installments_std,rate_new/hist_purchase_date_max,rate_new/hist_month_diff_mean,rate_new/hist_purchase_amount_sum,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean,first_active_month,outlier_flag
0,C_ID_0ab67a22ab,3,3,1,306.0,24.0,1.513186e+09,-0.476731,1.503441e+09,13.0,...,1.666667,1.666667,NaN,1.004401,1.000000,0.140001,-6.388255,1.120009,0.501563,0
1,C_ID_130fd0cbdd,2,3,0,396.0,1.0,1.494079e+09,-0.731881,1.494079e+09,1.0,...,1.000000,1.285714,0.202031,1.020192,-inf,8.121995,0.691844,0.902444,0.194557,0
2,C_ID_b709037bc5,5,1,1,184.0,4.0,1.517593e+09,0.214624,1.515681e+09,0.0,...,1.000000,10.000000,6.363961,1.002210,0.500000,0.033132,0.358015,0.066264,0.917728,0
3,C_ID_d27d835a9f,2,1,0,62.0,NaN,NaN,NaN,NaN,NaN,...,1.888889,12.000000,7.149635,NaN,NaN,NaN,NaN,NaN,1.333892,0
4,C_ID_2b5e3df5c2,5,1,1,793.0,23.0,1.514992e+09,-0.596643,1.491330e+09,0.0,...,0.833333,0.833333,NaN,1.005639,3.833333,-0.775375,39.511246,-2.972271,-1.159684,0


In [33]:

# test = test.drop(test.columns[0],axis=1)
# # 外れ値だけ他に置いておく
# test1 = test.query('outlier_flag == 1')
# # 外れ値でないものを抽出
# test = test.query('outlier_flag == 0')
# del test['outlier_flag']

## 保存されたモデルを使って予測を行う

In [70]:
n_folds = 5
predictions = np.zeros(len(test))
features = [c for c in test.columns if c not in ['card_id', 'first_active_month']]

for fold_ in range(n_folds):
    # モデルを読み込む
    with open(f'../src/models/modelq3-0_fold_{fold_}.pkl', 'rb') as f:
        clf = pickle.load(f)
    
    # モデルで予測を行う
    fold_predictions = clf.predict(test[features])
    
    # 予測結果を足し合わせる
    predictions += fold_predictions

# 平均を取る
predictions /= n_folds

# RMSEを計算する（テストデータに対して真の値がある場合）
# y_testが存在する場合に限り
if 'y_test' in locals():
    score = mean_squared_error(y_test, predictions, squared=False)
    print(f'RMSE={score}')

In [71]:
# 予測結果を出力する
print(predictions)

[-0.547759   -0.26936199 -0.57425688 ...  1.06049926 -0.85658671
  0.22483271]


外れ値を定数で予測

In [72]:
sub1_df = pd.DataFrame({"card_id":test1["card_id"].values})
sub1_df['target'] = np.zeros(len(test1)) + (- 33.2193)
sub1_df

,card_id,target
0,C_ID_c97d368d54,-33.2193
1,C_ID_c77547481e,-33.2193
2,C_ID_17a6bf702a,-33.2193
3,C_ID_ad81bfb0e2,-33.2193
4,C_ID_b4288eb949,-33.2193
...,...,...
150,C_ID_910dc70957,-33.2193
151,C_ID_5c70509b78,-33.2193
152,C_ID_8c7abb63fb,-33.2193
153,C_ID_79c453fb2a,-33.2193


## 提出用ファイルの作成

In [73]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions

# 予測と定数を結合
sub_df = pd.concat([sub_df,sub1_df], axis = 0)
sub_df.to_csv("submit_q3-0.csv", index=False)